# Train VAE to evaluate counterfactual realism
In this notebook, we train a VAE on the MNIST dataset so that we can evaluate the realism of any given counterfactual

## Setup


Import libraries

In [ ]:
import importlib
import regene_models
importlib.reload(regene_models)
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import new_CLUE
importlib.reload(new_CLUE)

<module 'new_CLUE' from '/Users/conor/Documents/College terms/College/Thesis/Thesis_Code_Minimised/MyImplementation/new_CLUE.py'>

Set the device

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


Load the Datasets

In [ ]:
# Load the MNIST dataset
from sklearn.model_selection import train_test_split

transform = transforms.Compose([transforms.ToTensor()])
mnist_dataset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform)

# Split the dataset into training and validation sets
train_indices, val_indices = train_test_split(np.arange(len(mnist_dataset)), test_size=0.2, random_state=42)

train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

trainloader = torch.utils.data.DataLoader(mnist_dataset, batch_size=64, sampler=train_sampler, num_workers=2)
valloader = torch.utils.data.DataLoader(mnist_dataset, batch_size=64, sampler=val_sampler, num_workers=2)

Create a models directory if it doesn't exist

In [ ]:
# Create models directory if it doesn't exist
os.makedirs('model_saves', exist_ok=True)

## Train

In [ ]:
from models.VAE_likelihood import train_vae_for_likelihood

vae = train_vae_for_likelihood(trainloader, valloader, device=device, epochs=100, patience=10, model_saves_dir='model_saves')